In [2]:
import json

import pandas as pd

from metric.fgcr_metric_cls import parse_instance

In [3]:
inputfile = 'data/fgcr_joint/test.json'
resultfile = 'model/fgcr/t5-small-debug2/predict_outputs.json'

In [4]:
with open(inputfile, 'r') as f:
    gold = json.load(f)
    gold = pd.DataFrame.from_dict(gold['data'])

with open(resultfile, 'r') as f:
    pred = json.load(f)
    pred = pd.DataFrame.from_dict(pred['predictions'])

In [5]:
gold

,context,question,question_type,answers,id
0,"BB&T and SunTrust have completed their merger,...",What are the events?,enable,[Cause] BB&T and SunTrust have completed their...,b6698f54
1,Given Tulip's lack of profitability (managemen...,What are the events?,enable,[Cause] Given Tulip's lack of profitability (m...,949978fd
2,TC Energy's profitability is not directly tied...,What are the events?,cause,[Cause] pipeline transportation costs are not ...,7969dd97
3,"That said, Wynn has recently renovated rooms a...",What are the events?,enable,[Cause] Wynn has recently renovated rooms and ...,f28f3868
4,We are keeping our fair value estimate for Dan...,What are the events?,cause,[Cause] recently generated cash flows and our ...,926ce9ce
...,...,...,...,...,...
2428,"Large global CROs, such as Syneos, have develo...",What are the events?,enable,[Cause] Large global CROs [Effect] produce the...,a6f86ef8
2429,In academic and professional learning (36% of ...,0 What are the events?,cause,[Cause] the COVID-19 outbreak forcing some cla...,1804f1f8
2430,In academic and professional learning (36% of ...,1 What are the events?,cause,[Cause] continued but moderating declines in p...,d0e06f21
2431,We expect the automotive segment carries less ...,What are the events?,cause,[Cause] hard-pressed motorists delay new vehic...,1e34a31a


In [6]:
data = pd.merge(gold, pred, on='id')
data.head()

,context,question,question_type,answers,id,prediction_text
0,"BB&T and SunTrust have completed their merger,...",What are the events?,enable,[Cause] BB&T and SunTrust have completed their...,b6698f54,[Cause] BB&T and SunTrust have completed their...
1,Given Tulip's lack of profitability (managemen...,What are the events?,enable,[Cause] Given Tulip's lack of profitability (m...,949978fd,[Cause] Tulip's lack of profitability [Effect]...
2,TC Energy's profitability is not directly tied...,What are the events?,cause,[Cause] pipeline transportation costs are not ...,7969dd97,[Cause] pipeline transportation costs are not ...
3,"That said, Wynn has recently renovated rooms a...",What are the events?,enable,[Cause] Wynn has recently renovated rooms and ...,f28f3868,[Cause] Wynn has recently renovated rooms and ...
4,We are keeping our fair value estimate for Dan...,What are the events?,cause,[Cause] recently generated cash flows and our ...,926ce9ce,[Cause] raising it in January 2021 [Effect] We...


In [9]:
for x in ['gold', 'pred']:
    label = {'gold': 'answers', 'pred': 'prediction_text'}[x]
    for y in ['Cause', 'Effect']:
        data[f'{x}_{y.lower()}'] = data[label].apply(lambda x: " | ".join(parse_instance(x)[0][y]))
    data[f'{x}_relation'] = data[label].apply(lambda x: parse_instance(x)[1])

data.head()

,context,question,question_type,answers,id,prediction_text,gold_Cause,gold_Effect,gold_Relation,pred_Cause,pred_Effect,pred_Relation,gold_cause,gold_effect,gold_relation,pred_cause,pred_effect,pred_relation
0,"BB&T and SunTrust have completed their merger,...",What are the events?,enable,[Cause] BB&T and SunTrust have completed their...,b6698f54,[Cause] BB&T and SunTrust have completed their...,"BB&T and SunTrust have completed their merger,...",which we believe will drive the next step up i...,enable,BB&T and SunTrust have completed their merger,drive the next step up in profitability for th...,enable,"BB&T and SunTrust have completed their merger,...",which we believe will drive the next step up i...,enable,BB&T and SunTrust have completed their merger,drive the next step up in profitability for th...,enable
1,Given Tulip's lack of profitability (managemen...,What are the events?,enable,[Cause] Given Tulip's lack of profitability (m...,949978fd,[Cause] Tulip's lack of profitability [Effect]...,Given Tulip's lack of profitability (managemen...,we do not believe the business maintains a cos...,enable,Tulip's lack of profitability,we do not believe the business maintains a cos...,cause,Given Tulip's lack of profitability (managemen...,we do not believe the business maintains a cos...,enable,Tulip's lack of profitability,we do not believe the business maintains a cos...,cause
2,TC Energy's profitability is not directly tied...,What are the events?,cause,[Cause] pipeline transportation costs are not ...,7969dd97,[Cause] pipeline transportation costs are not ...,pipeline transportation costs are not tied to ...,TC Energy's profitability is not directly tied...,cause,pipeline transportation costs are not tied to ...,TC Energy's profitability is not directly tied...,cause,pipeline transportation costs are not tied to ...,TC Energy's profitability is not directly tied...,cause,pipeline transportation costs are not tied to ...,TC Energy's profitability is not directly tied...,cause
3,"That said, Wynn has recently renovated rooms a...",What are the events?,enable,[Cause] Wynn has recently renovated rooms and ...,f28f3868,[Cause] Wynn has recently renovated rooms and ...,Wynn has recently renovated rooms and the gami...,help the facility maintain market share over t...,enable,Wynn has recently renovated rooms and the gami...,help the facility maintain market share over t...,enable,Wynn has recently renovated rooms and the gami...,help the facility maintain market share over t...,enable,Wynn has recently renovated rooms and the gami...,help the facility maintain market share over t...,enable
4,We are keeping our fair value estimate for Dan...,What are the events?,cause,[Cause] recently generated cash flows and our ...,926ce9ce,[Cause] raising it in January 2021 [Effect] We...,recently generated cash flows and our higher n...,We are keeping our fair value estimate for Dan...,cause,raising it in January 2021,We are keeping our fair value estimate for Dan...,cause,recently generated cash flows and our higher n...,We are keeping our fair value estimate for Dan...,cause,raising it in January 2021,We are keeping our fair value estimate for Dan...,cause


In [10]:
focus = data[['gold_cause', 'gold_effect', 'pred_cause', 'pred_effect', 'gold_relation', 'pred_relation']].copy()
focus.head()

,gold_cause,gold_effect,pred_cause,pred_effect,gold_relation,pred_relation
0,"BB&T and SunTrust have completed their merger,...",which we believe will drive the next step up i...,BB&T and SunTrust have completed their merger,drive the next step up in profitability for th...,enable,enable
1,Given Tulip's lack of profitability (managemen...,we do not believe the business maintains a cos...,Tulip's lack of profitability,we do not believe the business maintains a cos...,enable,cause
2,pipeline transportation costs are not tied to ...,TC Energy's profitability is not directly tied...,pipeline transportation costs are not tied to ...,TC Energy's profitability is not directly tied...,cause,cause
3,Wynn has recently renovated rooms and the gami...,help the facility maintain market share over t...,Wynn has recently renovated rooms and the gami...,help the facility maintain market share over t...,enable,enable
4,recently generated cash flows and our higher n...,We are keeping our fair value estimate for Dan...,raising it in January 2021,We are keeping our fair value estimate for Dan...,cause,cause


In [11]:
focus['correct'] = focus['gold_relation'] == focus['pred_relation']
focus.correct.value_counts()

True     1666
False     767
Name: correct, dtype: int64

In [12]:
focus['em'] = (focus['gold_cause'] == focus['pred_cause']) & (focus['gold_effect'] == focus['pred_effect'])
focus.em.value_counts()

False    1788
True      645
Name: em, dtype: int64

In [13]:
focus.loc[focus.correct == True, 'em'].value_counts(normalize=True)

False    0.705882
True     0.294118
Name: em, dtype: float64

In [14]:
focus.loc[focus.correct == False, 'em'].value_counts(normalize=True)

False    0.797914
True     0.202086
Name: em, dtype: float64

In [26]:
focus[focus.gold_relation == 'prevent'].correct.value_counts(normalize=True)

False    0.969466
True     0.030534
Name: correct, dtype: float64

In [27]:
from collections import defaultdict


matrix = defaultdict(int)

for row in focus.itertuples():
    matrix[(row.gold_relation, row.pred_relation)] += 1

labels = ['cause', 'enable', 'prevent']
print(f'{" ":8s} {"C":>5s} {"E":>5s} {"P":>5s}')

for l1 in labels:
    print(f'{l1:8s}', end=' ')
    for l2 in labels:
        print(f'{matrix[(l1, l2)]:5d}', end=' ')
    print()

             C     E     P
cause     1403   148     6 
enable     483   259     3 
prevent    120     7     4 
